In [1]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="p0Ximdf49v5ZJR5e4kPh")
project = rf.workspace("elephantdataset1").project("vid-f-eyjaz")
version = project.version(1)
dataset = version.download("yolov12")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.7/88.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 116.5 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to vid-f-1 in yolov12:: 100%|██████████| 220/220 [00:00<00:00, 5388.59it/s]


In [20]:
import os
import pandas as pd

# Your given helper function
def get_frame_from_filepath(file_path):
    """
    Extracts the frame number from a given file path.
    Example filename: Rec_20210304_150028_411_M_mp4-0216_jpg.rf.abc123.txt
    Returns: "0216"
    """
    file_name = os.path.basename(file_path)
    base_name = file_name.split('_jpg')[0]
    frame_number = base_name.split('-')[-1]
    return frame_number


# Your YOLO → (x1, y1, x2, y2) converter
def yolo_to_xyxy(label_path, img_width=1920, img_height=1080):
    with open(label_path, "r") as f:
        line = f.readline().strip().split()

    _, x_c, y_c, w, h = map(float, line)

    x_c *= img_width
    y_c *= img_height
    w *= img_width
    h *= img_height

    x1 = int(x_c - w / 2)
    y1 = int(y_c - h / 2)
    x2 = int(x_c + w / 2)
    y2 = int(y_c + h / 2)

    return x1, y1, x2, y2


# New function to create DataFrame from all label files
def create_dataframe_from_labels(label_dir):
    """
    Iterates through all YOLO label files in a directory,
    extracts frame numbers and bounding boxes,
    and creates a pandas DataFrame.

    Args:
        label_dir (str): Path to the directory with YOLO label .txt files

    Returns:
        pd.DataFrame: Columns = ["frame", "x1", "y1", "x2", "y2"]
    """
    data = []

    for file in os.listdir(label_dir):
        if file.endswith(".txt"):
            file_path = os.path.join(label_dir, file)

            frame = get_frame_from_filepath(file_path)
            x1, y1, x2, y2 = yolo_to_xyxy(file_path)

            data.append({"frame": frame, "x1": x1, "y1": y1, "x2": x2, "y2": y2})

    df = pd.DataFrame(data)
    return df

label_dir = "/content/vid-f-1/train/labels"
df = create_dataframe_from_labels(label_dir)
# Convert frame to integer for correct numeric sorting
df["frame"] = df["frame"].astype(int)* 50

# Sort by frame number
df = df.sort_values(by="frame").reset_index(drop=True)
df.head(10)

,frame,x1,y1,x2,y2
0,10800,925,87,986,123
1,10850,923,80,981,123
2,10900,912,95,976,138
3,10950,913,90,976,134
4,11000,911,89,976,142
5,11050,915,93,981,136
6,11100,917,85,984,134
7,11150,917,92,984,135
8,11200,917,93,987,135
9,11250,925,89,986,138


In [21]:
# Suppose df is your DataFrame
csv_path = "labels_xyxy.csv"
df.to_csv(csv_path, index=False)